# 1. Introduction

we will see how to integrate **multiple gRPC services** with a **Flask** web application. The goal is to demonstrate how data can **flow between services** (via **gRPC calls**) in a sequence. Flask will act as the **orchestrator**, making calls to each service and returning a **combined result**.

# 2. Setting Up gRPC

In [ ]:
pip install grpcio grpcio-tools flask

- **grpcio**: The core gRPC library.
- **grpcio-tools**: Tools to compile Protocol Buffers.
- **flask**: The Flask web framework for exposing HTTP endpoints.

# 3. Flask Application Setup

Create a **Flask** application that will expose an **HTTP API**. The Flask app will interact with **multiple gRPC services**, each performing part of a task.

## Directory Structure

.<br>
├── app.py              # Flask app to expose HTTP APIs<br>
├── grpc_server1.py     # gRPC server for Service 1<br>
├── grpc_server2.py     # gRPC server for Service 2<br>
├── service.proto       # Protobuf file defining the gRPC services<br>
├── service_pb2.py      # Generated Python file from the .proto file<br>
├── service_pb2_grpc.py # Generated Python file for gRPC services<br>

# 4. Creating Multiple gRPC Services

We will create two gRPC services:
- **Service 1**: Accepts a name and returns a greeting message.
- **Service 2**: Accepts a greeting and returns a transformed version (e.g., uppercased).

#### service.proto

In [ ]:
syntax = "proto3";

package myservice;

// The first gRPC service definition
service Service1 {
  rpc GetGreeting (GreetingRequest) returns (GreetingResponse) {
    option (grpc.http_compression) = true;  // Enable compression
  }
}

// The second gRPC service definition with streaming
service Service2 {
  rpc ModifyGreeting (stream GreetingRequest) returns (stream GreetingResponse);
}

// The request message containing the user's name.
message GreetingRequest {
  string name = 1;
}

// The response message containing the greeting.
message GreetingResponse {
  string message = 1;
}

Here, we define two services:
- Service1 with the **GetGreeting** method supports compression.
- Service2 with the **ModifyGreeting** method supports **bi-directional** streaming, where both the client and the server send/receive streams of messages.

#### For more details about Protocol Buffers see **Process Files/Protobuf.ipynb**

# 5. Building the gRPC Servers

We will implement each service as a **separate gRPC server**. Each server will handle its respective method.

#### grpc_server1.py (Service 1) (with compression)

In [ ]:
import grpc
from concurrent import futures
import service_pb2
import service_pb2_grpc

class Service1(service_pb2_grpc.Service1Servicer):
    def GetGreeting(self, request, context):
        """Service 1: Returns a greeting based on the user's name."""
        response = service_pb2.GreetingResponse()
        response.message = f"Hello, {request.name}!"
        return response

def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10),
                         options=[('grpc.default_compression_algorithm', grpc.Compression.Gzip)])  # Enable Gzip compression
    service_pb2_grpc.add_Service1Servicer_to_server(Service1(), server)
    server.add_insecure_port('[::]:50051')
    print("Service 1: gRPC server with compression started on port 50051")
    server.start()
    server.wait_for_termination()

if __name__ == '__main__':
    serve()

- **Explanation**:
The gRPC server for Service 1 now enables **Gzip compression** on all outgoing messages.


#### grpc_server2.py (Service 2)(with streaming)

In [ ]:
import grpc
from concurrent import futures
import service_pb2
import service_pb2_grpc

class Service2(service_pb2_grpc.Service2Servicer):
    def ModifyGreeting(self, request_iterator, context):
        for request in request_iterator:
            response = service_pb2.GreetingResponse()
            response.message = request.name.upper()  # Uppercase the greeting
            yield response  # Stream each response back to the client

def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    service_pb2_grpc.add_Service2Servicer_to_server(Service2(), server)
    server.add_insecure_port('[::]:50052')
    print("Service 2: gRPC server with streaming started on port 50052")
    server.start()
    server.wait_for_termination()

if __name__ == '__main__':
    serve()

- **Explanation**:
The gRPC server for Service 2 now supports **bi-directional streaming**. It processes each streamed request, modifies the message, and streams back the response.

- **Service 1** listens on port 50051 and returns a greeting message.
- **Service 2** listens on port 50052 and modifies the greeting (e.g., capitalizes it).

# 6. Connecting Flask with Multiple gRPC Services

Now, we’ll integrate these services with Flask. The Flask app will call both services in sequence:
- Call Service 1 to get a greeting message.
- Call Service 2 to modify the greeting.

#### app.py

In [ ]:
from flask import Flask, jsonify, request
import grpc
import service_pb2
import service_pb2_grpc

app = Flask(__name__)

# Function to get a gRPC stub for Service 1 with compression
def get_service1_stub():
    channel = grpc.insecure_channel('localhost:50051')
    stub = service_pb2_grpc.Service1Stub(channel)
    return stub

# Function to get a gRPC stub for Service 2 with streaming
def get_service2_stub():
    channel = grpc.insecure_channel('localhost:50052')
    stub = service_pb2_grpc.Service2Stub(channel)
    return stub

@app.route('/process_greeting', methods=['GET'])
def process_greeting():
    name = request.args.get('name', 'World')  # Default to 'World' if no name is provided
    
    # Step 1: Call Service 1 (GetGreeting with compression)
    service1_stub = get_service1_stub()
    request_message = service_pb2.GreetingRequest(name=name)
    response1 = service1_stub.GetGreeting(request_message, compression=grpc.Compression.Gzip)
    
    # Step 2: Call Service 2 (ModifyGreeting with streaming)
    service2_stub = get_service2_stub()
    request_message_stream = (service_pb2.GreetingRequest(name=response1.message) for _ in range(5))
    
    # Collect streamed responses
    responses = []
    for response in service2_stub.ModifyGreeting(request_message_stream):
        responses.append(response.message)
    
    # Return both the original and modified (streamed) greetings
    return jsonify({"original_greeting": response1.message, "modified_greetings": responses})

if __name__ == '__main__':
    app.run(port=5000)

### Explanation:
- **Step 1**: The Flask app calls Service 1 (on localhost:50051) to get a greeting message.
- **Step 2**: The Flask app then calls Service 2 (on localhost:50052) to modify the greeting (e.g., make it uppercase).
- **Final Result**: The Flask app returns both the original greeting and the modified greeting as a JSON response.

# 7. Running the Application

### Start Service 1:
Open a terminal and run:

In [ ]:
python grpc_server1.py

This will start Service 1 on localhost:50051.

### Start Service 2:
In another terminal, run:

In [ ]:
python grpc_server2.py

This will start Service 2 on localhost:50052.

### Start Flask:
In a third terminal, run:

python app.py

This will start the Flask app on localhost:5000.

# 8. Testing the Application

- To test the application, navigate to:<pre>
http://localhost:5000/process_greeting?name=John

- Expected Response:

In [ ]:
# Json
{
  "original_greeting": "Hello, John!",
  "modified_greetings": [
    "HELLO, JOHN!",
    "HELLO, JOHN!",
    "HELLO, JOHN!",
    "HELLO, JOHN!",
    "HELLO, JOHN!"
  ]
}

This response shows both the original greeting and the modified greeting after calling the two gRPC services.

# 9. Configuring Service Communication in Hosting (OnRender)

Since gRPC services are now hosted on Render and Flask is also deployed there, the services can communicate with each other using **internal URLs**:
- **Service 1**: service1.onrender.com:50051
- **Service 2**: service2.onrender.com:50052

In the Flask app, the communication flows as follows:
- The user sends a request to the Flask app hosted on flaskapp.onrender.com.
- Flask calls Service 1 via service1.onrender.com:50051 to get a greeting message.
- Flask then calls Service 2 via service2.onrender.com:50052 to modify the greeting.
- Finally, Flask returns the original and modified greetings to the user.